In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

#scene_dataset = "C:\\Users\\MatyuGuerrero\\Desktop\\Python\\Senior Project\\scenes" what data set is doing
scene_dataset = 'scenes' #put the filepath for the dataset
dataset_path = os.listdir(scene_dataset)
scene_types = os.listdir(scene_dataset)
print("The Scene Types are: ", scene_types) #what kind of scnene images are in this data set
print("Scenes types found:", len(scene_types))

In [ ]:
scenes = []

for item in scene_types: #This first for loop enters in our folder
    all_scenes = os.listdir(scene_dataset + '/' + item) #Gets all the file names
    #print(all_scenes)

    for scene in all_scenes: #This second list will get all the images of our folder
        scenes.append((item, str(scene_dataset + '/' + item) + '/' + scene)) 
        #print(scenes[:1])
scenes

In [ ]:
#build a data frame

scenes_dataframe = pd.DataFrame(data = scenes, columns = ['scene type', 'image']) #scene type shows the class label
print(scenes_dataframe.head())
#print(scenes_dataframe.tail())

In [ ]:
#check the size of the data set jtsu to see if balanced or unbalanced

print("Total number of scenes in the dataset:", len(scenes_dataframe))

scene_count = scenes_dataframe['scene type'].value_counts()

print("scnenes in each category:")
print(scene_count)

In [ ]:
import cv2

new_path = 'scenes/'

img_size = 224

images = []
labels = []

for i in scene_types:
    data_path =  new_path + str(i) #uhhhhhhhhh it works?
    filenames = [i for i in os.listdir(data_path)]
    #print(filenames) #get the names of all the imgs

    for f in filenames:
        img = cv2.imread(data_path + '/' + f)
        
        #catches 'bad' imgs 
        if img is None:
            print(f"Error loading jpg: {data_path}/{f}")
            continue
        
        #print(img) #will get img as an array
        img = cv2.resize(img, (img_size, img_size))
        images.append(img)
        labels.append(i)

labels

In [ ]:
#transform the image to a numpy type

images = np.array(images)
images.shape 
#prints out (total number of imgs, the length size, the width size, the number of channels (rgb thus 3))

In [ ]:
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

y = scenes_dataframe['scene type'].values
#print(y[:5])

#for y 
y_labelencoder = LabelEncoder ()
y_encoded = y_labelencoder.fit_transform (y)
print(y)
#print(y_encoded)


y = y.reshape(-1, 1)

ct = ColumnTransformer([('OneHotEncoder', OneHotEncoder(), [0])], remainder = 'passthrough')

Y = ct.fit_transform(y)
Y.shape
print(Y)


In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

images, Y = shuffle(images, Y, random_state = 1)

train_x, test_x, train_y, test_y = train_test_split(images, Y, test_size = 0.05, random_state = 415)

#impact the shape of the training and testing
print(train_x.shape) #(number of images)
print(train_y.shape)


print(test_x.shape) #(number to test)
print(test_y.shape)


In [ ]:
import numpy
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from tensorflow.python.keras.utils import layer_utils
from tensorflow.python.keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.optimizers import Adam
from keras.optimizers.schedules import ExponentialDecay
from keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix

import itertools

import pydot 
from IPython.display import SVG
from tensorflow.python.keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform

import scipy.misc
from matplotlib.pyplot import imshow
from keras.initializers import glorot_uniform
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D

In [ ]:
# x is input, y=F(x)
# identity block simply means input should be equal to output. 
#  y = x + F(x)   the layers in a traditional network are learning the true output H(x)
# F(x) = y - x   the layers in a residual network are learning the residual F(x)
# Hence, the name: Residual Block.



def identity_block(X, f, filters, stage, block):
    """
   
    Arguments:
    X -- input of shape (m, height, width, channel)
    f -- shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Saving the input value.we need this later to add to the output. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    
    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation 
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [ ]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    # First layer 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a')(X) # 1,1 is filter size
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)  # normalization on channels
    X = Activation('relu')(X)

      
    # Second layer  (f,f)=3*3 filter by default
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)


    # Third layer
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)


    ##### SHORTCUT PATH #### 
    X_shortcut = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1')(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value here, and pass it through a RELU activation 
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [ ]:
#Each ResNet block is either 2 layer deep
def ResNet50(input_shape=(64, 64, 3), classes=3):
    """
    Implementation of the ResNet50 architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input) #3,3 padding

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(X) #64 filters of 7*7 
    X = BatchNormalization(axis=3, name='bn_conv1')(X) #batchnorm applied on channels
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X) #window size is 3*3

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    # convolutional_block is a function defined above. Convolutional_block have 3 layers.
    #filters=[64, 64, 256] first 64 is for 1st layer and 2nd 64 is for 2nd layer and 256 is for 3rd layer of convultional block   
    # below are the conv layers from convolutional_block function
    #X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a')(X)
    #X = Conv2D(F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b')(X)
    #X = Conv2D(F3, (1, 1), strides = (s,s), name = conv_name_base + '2a')(X)
   
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b') 
    #X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a')(X)
    #X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b')(X)
    #X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c')(X)
  
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')
    #X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a')(X)
    #X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b')(X)
    #X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c')(X)


    ### START CODE HERE ###

    # Stage 3 
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4 
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5 
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL 
    X = AveragePooling2D((2,2), name="avg_pool")(X)

    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [ ]:
model = ResNet50(input_shape = (224, 224, 3), classes = 2)

In [ ]:
epochs = 3
batch_size = 32

In [ ]:
init_lr_rate = 0.1
lr_schedule = ExponentialDecay(init_lr_rate, decay_steps = 100000, decay_rate = 0.9, staircase = True)
optimizer = Adam(learning_rate = lr_schedule)

In [ ]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
#binary_crossentropy measures the difference between true labels and predicted prob for classification problems
#categorical_crossentropy when you have a multi-class classification

In [ ]:
early_stopping = EarlyStopping(monitor = 'val_accuracy', min_delta =.05, patience = 5, verbose = 1, mode = 'auto', restore_best_weights=True)

In [ ]:
model.summary()

In [ ]:
training_history = model.fit(train_x, train_y, epochs = epochs, batch_size = batch_size, validation_data = (test_x, test_y), callbacks = [early_stopping])

In [ ]:
preds = model.evaluate(test_x, test_y)
print("Loss = " + str(preds[0]))
print("Test Accuracy = " + str(preds[1]))

In [ ]:
model.save('model_resnet50live_run.keras')

best_acc = max(training_history.history['val_accuracy'])
print(f'best accuracy: {best_acc*100}')

In [ ]:
fig, ax = plt.subplots(2,1)
fig.suptitle('Learning Curves')

ax[0].plot(training_history.history['loss'], color ='b', label ="Training loss")
ax[0].plot(training_history.history['val_loss'], color ='r', label = "validation loss")
legend = ax[0].legend(loc ='best', shadow = True)
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Loss')


ax[1].plot(training_history.history['accuracy'], color ='b', label ="Training accuracy")
ax[1].plot(training_history.history['val_accuracy'], color ='r', label ="Validation accuracy")
legend = ax[1].legend(loc ='best', shadow = True)
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Accuracy')

plt.tight_layout()

In [ ]:
def plot_confusion_matrix(cm, classes, normalize = False, title ='Confusion Matrix', cmap = plt.cm.Blues):
  """
  This function prints and plots the confusion matrix.
  Normalization can be applied by setting normalize = True.
  """
  plt.imshow(cm, interpolation = 'nearest', cmap = cmap)
  #cm is an array rep the img to be displayed
  #interpolation:use nearest method for calculating pixel values when resizing
  #cmap: parameter used to specify color scale to be used for mapping scalar data to colors
  plt.title(title)
  plt.colorbar()
  #configure tick marks based on length of classes array
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation = 45)
  plt.yticks(tick_marks, classes)

  if normalize:
    cm = cm.astype('float') / cm.sum(axis = 1)[:, np.newaxis]
  thresh = cm.max() / 2.

  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')

In [ ]:
# Predict the values from the validation dataset
Y_pred = model.predict(test_x)
Y_pred_classes = np.argmax(Y_pred, axis = 1)
Y_true = np.argmax(test_y, axis = 1)

confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
plot_confusion_matrix(confusion_mtx, classes = range(2))

In [ ]:
import cv2
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from matplotlib.pyplot import imshow

def camera_frame(frame):
    frame = cv2.resize(frame, (224,224)) #resize the current frame to 224,224
    x = image.img_to_array(frame)
    x = np.expand_dims(x, axis = 0)
    x = preprocess_input(x) 
    return x

capture = cv2.VideoCapture(0)

while True:
    ret, frame = capture.read()
    if not ret:
        print("Failed to capture frame")
        break
    
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1)

    if key == 32:

        x = camera_frame(frame)
        prediction = model.predict(x)
        print("Prediction: ", prediction)

        imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        plt.show()

        
    if key == ord('q'): #press q to quit
        break

capture.release()
cv2.destroyAllWindows()
